In [3]:
# Notwendige Importe
import pandas as pd
import sqlalchemy as sql
import matplotlib.pyplot as plt
import seaborn as sns

# Verbindung zur PostgreSQL-Datenbank herstellen
db_user = 'postgres'
db_password = 'neues_passwort'
db_host = 'localhost'
db_port = '5432'
db_name = 'postgres'

# Engine erstellen
engine = sql.create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Funktion zum Laden eines DataFrames aus der PostgreSQL-Datenbank
def load_table_from_db(table_name, engine):
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql(query, engine)
    return df

# Beispiel: Laden der Faktentabelle
faktentabelle_df = load_table_from_db('faktentabelle', engine)

# Beispiel: Laden einer Dimensionstabelle
dim_land_df = load_table_from_db('dim_land', engine)

# Erste Zeilen anzeigen, um die Daten zu überprüfen
print("Faktentabelle:")
print(faktentabelle_df.head())

print("\nDimensionstabelle - Land:")
print(dim_land_df.head())



Faktentabelle:
   faktenschluessel  landesschluessel gemeindeschluessel  \
0                 1                10               None   
1                 2                 3               None   
2                 3                12               None   
3                 4                 5               None   
4                 5                 1               None   

  gemeindeverbandsschluessel regbezirkschluessel kreisschluessel  \
0                       None                None            None   
1                       None                None            None   
2                       None                None            None   
3                       None                None            None   
4                       None                None            None   

  stadttypschluessel  gesamtbevölkerung  erwerbstätige  arbeitslose  \
0               None                  0              0            0   
1               None                  0              0            0   
2 

OSError: 'seaborn-darkgrid' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)